In [1]:
from warnings import filterwarnings
filterwarnings("ignore")

In [2]:
import pandas as pd
A = pd.read_csv("C:/Users/kruta/Downloads/train.csv")
B = pd.read_csv("C:/Users/kruta/Downloads/test.csv")

Find for missing values

In [3]:
A.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [4]:
B.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [5]:
for i in A.columns:
    if (A[i].dtypes == "object"):
        x = A[i].mode()[0]
        A[i] = A[i].fillna(x)
    else:
        x = A[i].mean()
        A[i] = A[i].fillna(x)
        
for i in B.columns:
    if (B[i].dtypes == "object"):
        x = B[i].mode()[0]
        B[i] = B[i].fillna(x)
    else:
        x = B[i].mean()
        B[i] = B[i].fillna(x)


In [6]:
A.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [7]:
B.isna().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

As checked above no missing values now

Checking the correlation of continous features with "Survived"

In [8]:
A.corr()[["Survived"]]

,Survived
PassengerId,-0.005007
Survived,1.000000
Pclass,-0.338481
Age,-0.069809
SibSp,-0.035322
Parch,0.081629
Fare,0.257307


Dropping the unnecessary features based on correlation, also dropping "Survived" feature as its not necessary to predict

In [9]:
X = A.drop(labels = ["PassengerId","Survived", "Pclass" , "Age","SibSp","Parch"],axis = 1)
Y = A[["Survived"]]

Seperating categorical and continous features to use min - max (Standardization)

In [10]:
cat = []
con = []

for i in X.columns:
    if (X[i].dtypes == "object"):
        cat.append(i)
    else:
        con.append(i)
        

In [11]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X1 = pd.DataFrame(ss.fit_transform(X[con]),columns = con)

Drop the outliers

In [12]:
for i  in X1.columns:
    outliers = []
    outliers.extend(list(X1[(X1[i] > 3) | (X1[i] < -3)].index))
    
import numpy as np
outliers = np.unique(outliers)

In [13]:
outliers

array([ 27,  88, 118, 258, 299, 311, 341, 377, 380, 438, 527, 557, 679,
       689, 700, 716, 730, 737, 742, 779])

In [14]:
X1 = X1.drop(index = outliers, axis = 0)

In [15]:
Y = Y.drop(index = outliers, axis = 0)

Now check the correlation of categorical features with "Survived" using ANOVA test

In [16]:
for i in cat:
    from statsmodels.formula.api import ols
    rel = "Survived ~" +str(i)
    model = ols(rel,A).fit()
    from statsmodels.stats.anova import anova_lm
    anova_results = anova_lm(model)
    print(rel, "-->", round(anova_results.iloc[0,4],4))

Survived ~Name --> nan
Survived ~Sex --> 0.0
Survived ~Ticket --> 0.0
Survived ~Cabin --> 0.0
Survived ~Embarked --> 0.0


No need to drop features based on categorical columns

In [17]:
cat.remove("Name")

One Hot Encoding

In [18]:
X2 = pd.get_dummies(X[cat])

In [19]:
Xnew = X1.join(X2)

In [20]:
Xnew.head()

,Fare,Sex_female,Sex_male,Ticket_110152,Ticket_110413,Ticket_110465,Ticket_110564,Ticket_110813,Ticket_111240,Ticket_111320,...,Cabin_F G73,Cabin_F2,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Embarked_C,Embarked_Q,Embarked_S
0,-0.502445,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0.786845,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,-0.488854,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0.420730,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,-0.486337,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [21]:
Xnew.shape

(871, 834)

In [22]:
Y.shape

(871, 1)

In [23]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size= 0.2,random_state=21)

In [24]:
Xnew.index = range(0,len(Xnew),1)

In [25]:
Y.index = range(0,len(Xnew),1)

In [26]:
for i in cat:
    from statsmodels.formula.api import ols
    rel = "Survived ~" + str(i)
    model = ols(rel,A).fit()
    from statsmodels.stats.anova import anova_lm
    anova_results = anova_lm(model)
    print(rel, "--> ",round(anova_results.iloc[0,4],4))

Survived ~Sex -->  0.0
Survived ~Ticket -->  0.0
Survived ~Cabin -->  0.0
Survived ~Embarked -->  0.0


In [27]:
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Survived   R-squared:                       0.919
Model:                            OLS   Adj. R-squared:                  0.528
Method:                 Least Squares   F-statistic:                     2.352
Date:                Mon, 04 Jul 2022   Prob (F-statistic):           2.97e-08
Time:                        22:08:42   Log-Likelihood:                 394.17
No. Observations:                 696   AIC:                             365.7
Df Residuals:                     119   BIC:                             2988.
Df Model:                         576                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                         0.3310      6.041      0.055      0.956     -11.631      12.293
Fare                       -1.15e-10     37.084   -3.1e-12      1.000     -73.430      73.430
Sex_female                    0.3348      3.021      0.111      0.912      -5.646       6.316
Sex_male                     -0.0038      3.021     -0.001      0.999      -5.985       5.977
Ticket_110152                 0.1811     20.272      0.009      0.993     -39.960      40.322
Ticket_110413                -0.0295     12.647     -0.002      0.998     -25.071      25.012
Ticket_110465                -0.2598      3.102     -0.084      0.933      -6.402       5.882
Ticket_110564             -1.264e-11   2.21e-09     -0.006      0.995   -4.38e-09    4.36e-09
Ticket_110813                 0.0950      7.766      0.012      0.990     -15.282      15.472
Ticket_111240                -0.1949      4.586     -0.042      0.966      -9.276       8.887
Ticket_111320                -0.1949      2.722     -0.072      0.943      -5.584       5.194
Ticket_111361                 0.0950      1.327      0.072      0.943      -2.532       2.722
Ticket_111369                 0.2643      9.131      0.029      0.977     -17.816      18.345
Ticket_111426                 0.7433      2.494      0.298      0.766      -4.195       5.681
Ticket_111427                 0.8249      8.957      0.092      0.927     -16.911      18.561
Ticket_111428                 0.8249      8.957      0.092      0.927     -16.911      18.561
Ticket_112050                -0.1949     17.091     -0.011      0.991     -34.037      33.647
Ticket_112052                -0.1751     10.878     -0.016      0.987     -21.716      21.365
Ticket_112053                 0.1358      5.892      0.023      0.982     -11.532      11.803
Ticket_112058                -0.1949     17.091     -0.011      0.991     -34.037      33.647
Ticket_112059                -0.1949     17.091     -0.011      0.991     -34.037      33.647
Ticket_112277                 0.2643      8.758      0.030      0.976     -17.077      17.606
Ticket_112379                -0.2567     12.220     -0.021      0.983     -24.455      23.941
Ticket_113028                -0.1299      5.277     -0.025      0.980     -10.578      10.318
Ticket_113043                -0.1299      3.823     -0.034      0.973      -7.699       7.439
Ticket_113050                -0.1949      7.180     -0.027      0.978     -14.412      14.022
Ticket_113051                -0.2357      9.971     -0.024      0.981     -19.979      19.508
Ticket_113055             -4.535e-16   1.71e-13     -0.003      0.998   -3.39e-13    3.38e-13
Ticket_113056                -0.1949      7.385     -0.026      0.979     -14.818      14.429
Ticket_113059                -0.1751     24.297     -0.007      0.994     -

In [28]:
cols_to_drop = model.pvalues.sort_values().index[-1]

In [29]:
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,4)

0.6201

In [30]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.6201

In [31]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.6201

In [32]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.6201

In [33]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.6201

In [34]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.6201

In [35]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.6201

In [36]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.6201

In [37]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.6201

In [38]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.6201

In [39]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.6201

In [40]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.6201

In [41]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.6201

In [42]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.6201

In [43]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.6201

In [44]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.6201

In [45]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.6201

In [46]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.6201

In [47]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.6201

In [48]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.6201

In [49]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.6201

In [50]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.6201

In [51]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.6201

In [52]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.6201

In [53]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.6201

In [54]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.6201

In [55]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.6201

In [56]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.6201

In [57]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.6201

In [58]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.6201

In [59]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.6201

In [60]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.623

In [61]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.623

In [62]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.623

In [63]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.623

In [64]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.623

In [65]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.623

In [66]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.623

In [67]:
cols_to_drop = model.pvalues.sort_values().index[-1]
Xnew = Xnew.drop(labels = cols_to_drop,axis=1)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=31)
from statsmodels.api import add_constant,OLS
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()
round(model.rsquared_adj,5)

0.60193

In [68]:
cols_to_drop

'Ticket_PC 17757'

In [69]:
Z = X2[["Ticket_PC 17757"]]

In [70]:
Xnew = Xnew.join(Z)

Check For Overfitting

In [71]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size= 0.2,random_state=21)

In [72]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
model = lm.fit(xtrain,ytrain)
tr_pred = model.predict(xtrain)
ts_pred = model.predict(xtest)

from sklearn.metrics import mean_squared_error
tr_err = round(mean_squared_error(ytrain,tr_pred),2)
ts_err = round(mean_squared_error(ytest,ts_pred),2)

In [73]:
tr_err

0.04

In [74]:
ts_err

4.514629056767072e+25

Reduce overfitting using ridge or lasso model

In [75]:
from sklearn.linear_model import Ridge,Lasso
rr = Ridge()
ls = Lasso()

In [76]:
q = 0.01
w = []
for i in range(0,300):
    w.append(q)
    q = q + 0.01

GridSearchCV for alpha value

In [77]:
from sklearn.model_selection import GridSearchCV

tuning_grid = {"alpha":w}
cv = GridSearchCV(rr,tuning_grid,scoring="neg_mean_squared_error",cv = 4)
cvmodel = cv.fit(Xnew,Y)
print(cvmodel.best_params_,"alpha value using Ridge")

{'alpha': 1.0300000000000007} alpha value using Ridge


In [78]:
from sklearn.model_selection import GridSearchCV

tuning_grid = {"alpha":w}
cv = GridSearchCV(ls,tuning_grid,scoring="neg_mean_squared_error",cv=4)
cvmodel = cv.fit(Xnew,Y)
print(cvmodel.best_params_ ,"alpha value using Lasso")

{'alpha': 0.01} alpha value using Lasso


In [79]:
rr = Ridge(alpha=1.03)
model = rr.fit(xtrain,ytrain)
tr_pred = model.predict(xtrain) 
ts_pred = model.predict(xtest)
tr_err = round(mean_squared_error(ytrain,tr_pred),2)
ts_err = round(mean_squared_error(ytest,ts_pred),2)
print(tr_err,ts_err)

0.05 0.12


In [80]:
ls = Lasso(alpha=0.01)
model = ls.fit(xtrain,ytrain)
tr_pred = model.predict(xtrain) 
ts_pred = model.predict(xtest)
tr_err = round(mean_squared_error(ytrain,tr_pred),2)
ts_err = round(mean_squared_error(ytest,ts_pred),2)
print(tr_err,ts_err)

0.16 0.14


Making Test Set Ready for Prediction

In [81]:
B.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,B57 B59 B63 B66,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,B57 B59 B63 B66,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,B57 B59 B63 B66,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,B57 B59 B63 B66,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,B57 B59 B63 B66,S


In [82]:
B.isna().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [83]:
X = B.drop(labels = ["PassengerId", "Pclass" , "Age","SibSp","Parch"],axis = 1)

In [84]:
cat = []
con = []
for i in X.columns:
    if(X[i].dtypes == "object"):
        cat.append(i)
    else:
        con.append(i)

In [85]:
Xtst1 = pd.DataFrame(ss.fit_transform(X[con]),columns = con)

In [86]:
cat.remove("Name")

In [87]:
Xtst2 = pd.get_dummies(B[cat])

In [88]:
Xtst_new = Xtst1.join(Xtst2)

In [89]:
cols_to_keep = xtrain.columns

In [90]:
Xtst_new.index = range(0,len(Xtst_new),1)

In [91]:
new_list = ['Cabin_B20', 'Ticket_220367', 'Ticket_2672', 'Cabin_B79', 'Ticket_342826', 'Cabin_E77', 'Ticket_STON/O 2. 3101275', 'Ticket_SC/Paris 2163', 'Ticket_SC/PARIS 2146', 'Ticket_C.A. 29178', 'Ticket_350052', 'Ticket_PC 17476', 'Cabin_C123', 'Ticket_374746', 'Ticket_349257', 'Ticket_PC 17609', 'Ticket_7534', 'Cabin_C110', 'Ticket_349237', 'Ticket_236852', 'Ticket_113800', 'Cabin_B18', 'Cabin_D56', 'Ticket_7267', 'Ticket_13568', 'Ticket_STON/O2. 3101283', 'Ticket_WE/P 5735', 'Ticket_312992', 'Ticket_394140', 'Ticket_364500', 'Ticket_STON/O 2. 3101273', 'Ticket_PC 17611', 'Ticket_349231', 'Ticket_PC 17601', 'Ticket_349217', 'Ticket_4579', 'Ticket_370370', 'Cabin_D17', 'Ticket_335097', 'Ticket_PC 17473', 'Ticket_A/5 2817', 'Ticket_113510', 'Ticket_112053', 'Ticket_110413', 'Ticket_113572', 'Ticket_343276', 'Ticket_345779', 'Ticket_2690', 'Ticket_345773', 'Ticket_SC/PARIS 2133', 'Ticket_28424', 'Ticket_111369', 'Ticket_PC 17610', 'Ticket_330979', 'Ticket_347083', 'Ticket_2695', 'Ticket_113789', 'Cabin_D7', 'Ticket_365222', 'Cabin_D48', 'Ticket_250649', 'Ticket_A.5. 11206', 'Ticket_349246', 'Cabin_E44', 'Ticket_A/5 3536', 'Cabin_A36', 'Ticket_315088', 'Ticket_2659', 'Ticket_14312', 'Ticket_PC 17600', 'Ticket_348123', 'Ticket_347089', 'Ticket_2663', 'Ticket_349204', 'Ticket_28206', 'Ticket_248747', 'Ticket_113803', 'Ticket_SC/PARIS 2149', 'Ticket_2664', 'Ticket_A/4. 20589', 'Cabin_B39', 'Ticket_3460', 'Ticket_347060', 'Ticket_239854', 'Ticket_334912', 'Ticket_3101296', 'Ticket_3101277', 'Ticket_347087', 'Ticket_Fa 265302', 'Cabin_A6', 'Ticket_11755', 'Ticket_243880', 'Ticket_349208', 'Ticket_35281', 'Ticket_343120', 'Ticket_343275', 'Ticket_36865', 'Ticket_234818', 'Cabin_C148', 'Ticket_PC 17474', 'Ticket_384461', 'Ticket_SO/C 14885', 'Ticket_SOTON/O.Q. 3101311', 'Ticket_348121', 'Ticket_13213', 'Ticket_330935', 'Cabin_D20', 'Ticket_113051', 'Ticket_SOTON/OQ 3101317', 'Ticket_28425', 'Ticket_3411', 'Ticket_244373', 'Ticket_250644', 'Ticket_370377', 'Cabin_E24', 'Ticket_237668', 'Ticket_237442', 'Cabin_B73', 'Cabin_E121', 'Cabin_C90', 'Ticket_350417', 'Ticket_2699', 'Ticket_SOTON/O.Q. 3101312', 'Ticket_2666', 'Ticket_S.O./P.P. 751', 'Ticket_239856', 'Ticket_347466', 'Ticket_SC/PARIS 2131', 'Ticket_347064', 'Ticket_349222', 'Ticket_695', 'Ticket_111428', 'Ticket_243847', 'Ticket_250653', 'Ticket_364846', 'Cabin_C50', 'Cabin_D6', 'Ticket_113807', 'Ticket_330980', 'Ticket_2641', 'Ticket_9234', 'Ticket_371110', 'Ticket_315093', 'Ticket_2693', 'Ticket_349251', 'Ticket_230080', 'Ticket_19972', 'Ticket_240929', 'Ticket_349236', 'Ticket_C 7077', 'Ticket_SOTON/OQ 392082', 'Ticket_349240', 'Ticket_347062', 'Ticket_349228', 'Ticket_364848', 'Ticket_SOTON/OQ 392076', 'Ticket_17453', 'Ticket_A.5. 18509', 'Ticket_13509', 'Cabin_B30', 'Ticket_113501', 'Ticket_330959', 'Ticket_315094', 'Ticket_STON/O 2. 3101292', 'Ticket_250652', 'Ticket_347076', 'Ticket_A/5 3902', 'Ticket_A/5. 10482', 'Ticket_A/5. 3336', 'Ticket_28134', 'Ticket_347464', 'Ticket_368703', 'Ticket_2623', 'Ticket_7540', 'Ticket_27267', 'Ticket_113776', 'Ticket_374910', 'Ticket_13049', 'Ticket_C.A. 29566', 'Cabin_A7', 'Cabin_D45', 'Ticket_345774', 'Ticket_347468', 'Ticket_315151', 'Ticket_2669', 'Cabin_E10', 'Ticket_370369', 'Ticket_11751', 'Ticket_112052', 'Ticket_11752', 'Ticket_349215', 'Ticket_2700', 'Cabin_C124', 'Ticket_STON/O 2. 3101294', 'Ticket_367231', 'Ticket_A./5. 2152', 'Ticket_4137', 'Ticket_349205', 'Ticket_237736', 'Ticket_STON/O 2. 3101274', 'Cabin_B38', 'Ticket_349216', 'Ticket_323951', 'Ticket_3101276', 'Ticket_367228', 'Ticket_370372', 'Cabin_A24', 'Ticket_315090', 'Cabin_C128', 'Ticket_C.A. 31026', 'Ticket_SOTON/O.Q. 3101306', 'Ticket_36973', 'Ticket_111427', 'Ticket_347071', 'Ticket_315084', 'Ticket_248723', 'Ticket_250648', 'Ticket_349203', 'Ticket_347074', 'Ticket_PC 17582', 'Ticket_3474', 'Ticket_345364', 'Cabin_D49', 'Cabin_C92', 'Cabin_A20', 'Cabin_E58', 'Ticket_110465', 'Ticket_13507', 'Ticket_336439', 'Ticket_315037', 'Ticket_341826', 'Ticket_248698', 'Cabin_B49', 'Ticket_392096', 'Ticket_113760', 'Cabin_C93', 'Ticket_345780', 'Ticket_C 17369', 'Ticket_364851', 'Ticket_12460', 'Ticket_345765', 'Ticket_2626', 'Ticket_244252', 'Ticket_364511', 'Ticket_A/5 3540', 'Ticket_350406', 'Ticket_31418', 'Cabin_F E69', 'Ticket_349233', 'Ticket_113505', 'Ticket_STON/O2. 3101290', 'Ticket_349224', 'Cabin_C103', 'Ticket_PC 17605', 'Ticket_4138', 'Ticket_349219', 'Ticket_383121', 'Ticket_350034', 'Ticket_28213', 'Ticket_PC 17572', 'Ticket_250647', 'Ticket_65306', 'Ticket_2694', 'Ticket_2677', 'Ticket_343095', 'Ticket_17474', 'Ticket_65304', 'Ticket_7598', 'Ticket_113783', 'Ticket_STON/O 2. 3101288', 'Ticket_2687', 'Ticket_349207', 'Ticket_C.A. 31921', 'Ticket_3101278', 'Ticket_A/5 2466', 'Ticket_CA. 2314', 'Cabin_A26', 'Cabin_E101', 'Ticket_A/5. 2151', 'Ticket_239865', 'Ticket_350035', 'Ticket_C.A. 33111', 'Ticket_28228', 'Ticket_350029', 'Ticket_6563', 'Ticket_5727', 'Ticket_350047', 'Ticket_113798', 'Ticket_349206', 'Cabin_E40', 'Ticket_239853', 'Ticket_2651', 'Ticket_113787', 'Ticket_349243', 'Ticket_7545', 'Cabin_C49', 'Ticket_250646', 'Ticket_PC 17754', 'Cabin_D26', 'Ticket_13214', 'Ticket_12233', 'Ticket_PC 17596', 'Cabin_C87', 'Ticket_349242', 'Cabin_B94', 'Ticket_218629', 'Ticket_29751', 'Cabin_C99', 'Ticket_365226', 'Ticket_54636', 'Ticket_367229', 'Ticket_330909', 'Ticket_PC 17595', 'Ticket_350404', 'Ticket_113055', 'Ticket_7553', 'Ticket_A/5. 13032', 'Ticket_4134', 'Ticket_250655', 'Ticket_349209', 'Ticket_244310', 'Ticket_PP 4348', 'Ticket_312991', 'Ticket_7552', 'Ticket_27042', 'Ticket_31028', 'Ticket_SOTON/OQ 3101316', 'Ticket_C.A. 29395', 'Cabin_E25', 'Ticket_A4. 54510', 'Ticket_113056', 'Ticket_349221', 'Ticket_2649', 'Ticket_C 7075', 'Cabin_D9', 'Cabin_D11', 'Ticket_SOTON/O2 3101287', 'Ticket_330923', 'Ticket_29106', 'Ticket_349253', 'Ticket_349225', 'Ticket_364499', 'Ticket_2671', 'Ticket_376564', 'Ticket_219533', 'Ticket_2629', 'Ticket_330931', 'Ticket_A/4 45380', 'Ticket_SW/PP 751', 'Ticket_17466', 'Ticket_362316', 'Ticket_4135', 'Cabin_C68', 'Ticket_364516', 'Ticket_233639', 'Ticket_112050', 'Ticket_250643', 'Ticket_364512', 'Ticket_113028', 'Ticket_2665', 'Cabin_B96 B98', 'Ticket_113804', 'Cabin_A10', 'Ticket_248731', 'Ticket_110152', 'Ticket_STON/O 2. 3101293', 'Ticket_363294', 'Ticket_223596', 'Ticket_244361', 'Ticket_C.A. 5547', 'Ticket_244367', 'Cabin_B82 B84', 'Ticket_PC 17590', 'Ticket_347061', 'Ticket_STON/O2. 3101282', 'Ticket_35852', 'Ticket_349210', 'Ticket_349218', 'Ticket_233866', 'Ticket_370365', 'Ticket_W./C. 6609', 'Ticket_19947', 'Ticket_349223', 'Cabin_C126', 'Ticket_112379', 'Ticket_PC 17318', 'Ticket_A/5 21173', 'Ticket_372622', 'Ticket_349241', 'Ticket_36209', 'Ticket_34218', 'Ticket_113767', 'Ticket_345767', 'Ticket_3101265', 'Ticket_382649', 'Ticket_2697', 'Ticket_349244', 'Ticket_345783', 'Ticket_A/S 2816', 'Ticket_234360', 'Ticket_13502', 'Ticket_345781', 'Ticket_14973', 'Ticket_315098', 'Ticket_347073', 'Ticket_LINE', 'Ticket_231945', 'Ticket_A/4. 39886', 'Cabin_E8', 'Ticket_19988', 'Ticket_11771', 'Ticket_2691', 'Ticket_113786', 'Ticket_C.A. 24580', 'Cabin_B77', 'Cabin_A31', 'Cabin_B28', 'Ticket_A/5 21171', 'Ticket_2647', 'Ticket_31027', 'Ticket_113514', 'Ticket_STON/O2. 3101271', 'Cabin_B19', 'Ticket_2685', 'Ticket_358585', 'Ticket_2908', 'Ticket_STON/O 2. 3101269', 'Ticket_349249', 'Cabin_B35', 'Ticket_347081', 'Ticket_248740', 'Ticket_STON/O 2. 3101286', 'Ticket_349239', 'Ticket_36967', 'Ticket_370375', 'Cabin_C91', 'Ticket_330919', 'Ticket_7546', 'Ticket_PC 17477', 'Cabin_C52', 'Ticket_36947', 'Ticket_F.C.C. 13531', 'Cabin_E49', 'Ticket_312993', 'Ticket_345777', 'Ticket_17764', 'Ticket_SC/AH Basle 541', 'Ticket_239855', 'Ticket_373450', 'Ticket_367232', 'Ticket_14313', 'Ticket_347742', 'Ticket_SOTON/O.Q. 3101310', 'Ticket_211536', 'Ticket_2620', 'Ticket_A/4. 34244', 'Ticket_PC 17593', 'Ticket_345778', 'Ticket_348124', 'Cabin_B22', 'Ticket_2686', 'Ticket_29104', 'Ticket_392092', 'Ticket_113043', 'Ticket_347088', 'Ticket_17421', 'Ticket_65303', 'Ticket_A/5 21172', 'Ticket_349214', 'Ticket_347054', 'Ticket_SOTON/O.Q. 392087', 'Ticket_35273', 'Ticket_2683', 'Ticket_248727', 'Ticket_693', 'Ticket_2667', 'Ticket_347470', 'Ticket_36864', 'Ticket_17465', 'Ticket_349254', 'Ticket_368323', 'Ticket_350046', 'Ticket_A/5 21174', 'Cabin_E36', 'Cabin_E67', 'Ticket_349252', 'Ticket_PC 17482', 'Ticket_350050', 'Ticket_350025', 'Ticket_8475', 'Ticket_3101267', 'Ticket_SOTON/O.Q. 392078', 'Ticket_113784', 'Ticket_C.A. 18723', 'Ticket_C.A./SOTON 34068', 'Ticket_349227', 'Ticket_SOTON/O.Q. 3101305', 'Ticket_2223', 'Ticket_A./5. 3235', 'Cabin_A16', 'Ticket_28551', 'Ticket_236171', 'Ticket_11668', 'Ticket_11967', 'Ticket_230434', 'Cabin_B102', 'Ticket_347063', 'Ticket_234604', 'Ticket_PC 17475', 'Ticket_SOTON/OQ 392090', 'Ticket_P/PP 3381', 'Ticket_347067', 'Ticket_S.O.P. 1166', 'Cabin_A5', 'Ticket_28403', 'Ticket_237565', 'Ticket_STON/O 2. 3101280', 'Ticket_21440', 'Cabin_C47', 'Ticket_370376', 'Ticket_2648', 'Ticket_111320', 'Ticket_382651', 'Cabin_C111', 'Ticket_315082', 'Cabin_A32', 'Ticket_S.C./A.4. 23567', 'Cabin_F G73', 'Ticket_C.A. 24579', 'Ticket_330877', 'Ticket_350042', 'Ticket_244270', 'Ticket_112277', 'Cabin_B50', 'Ticket_SOTON/OQ 392089', 'Ticket_36963', 'Ticket_2674', 'Ticket_350060', 'Ticket_SC 1748', 'Ticket_350036', 'Ticket_345764', 'Ticket_349234', 'Ticket_112059', 'Ticket_364506', 'Ticket_C.A. 34260', 'Ticket_113788', 'Ticket_111361', 'Cabin_A19', 'Ticket_330958', 'Ticket_35851', 'Cabin_A23', 'Ticket_347082', 'Ticket_14311', 'Ticket_STON/O 2. 3101285', 'Ticket_324669', 'Ticket_36866', 'Ticket_113794', 'Cabin_D33', 'Ticket_4136', 'Cabin_D35', 'Ticket_C.A. 17248', 'Ticket_230433', 'Ticket_349245', 'Ticket_SOTON/O2 3101272', 'Ticket_244278', 'Ticket_349248', 'Ticket_113792', 'Ticket_363592', 'Ticket_350407', 'Ticket_8471', 'Ticket_SCO/W 1585', 'Ticket_3101281', 'Ticket_STON/O2. 3101279', 'Ticket_330932', 'Ticket_350048', 'Ticket_2628', 'Ticket_237798', 'Ticket_347743', 'Ticket_SOTON/OQ 392086', 'Ticket_19996', 'Ticket_349212', 'Ticket_16988', 'Ticket_W./C. 14263', 'Cabin_D47', 'Ticket_347068', 'Ticket_248706', 'Ticket_364850', 'Ticket_S.W./PP 752', 'Ticket_SOTON/O.Q. 3101307', 'Ticket_371060', 'Ticket_11774', 'Ticket_226593', 'Cabin_A14', 'Cabin_C83', 'Ticket_PC 17485', 'Ticket_345770', 'Ticket_349247', 'Ticket_S.P. 3464', 'Ticket_347078', 'Ticket_29011', 'Ticket_19943', 'Ticket_345769', 'Ticket_347069', 'Ticket_363291', 'Ticket_349912', 'Cabin_B37', 'Cabin_E33', 'Ticket_2627', 'Ticket_234686', 'Cabin_D36', 'Ticket_367230', 'Ticket_27849', 'Ticket_323592', 'Ticket_347085', 'Ticket_C 7076', 'Ticket_3101264', 'Ticket_349201', 'Ticket_367655', 'Ticket_W./C. 14258', 'Ticket_315097', 'Ticket_364849', 'Ticket_28665', 'Ticket_W/C 14208', 'Ticket_248733', 'Ticket_111240', 'Ticket_19952', 'Ticket_374887', 'Cabin_B42', 'Ticket_PC 17612', 'Ticket_29108', 'Ticket_A/5 3594', 'Ticket_STON/O 2. 3101289', 'Ticket_113050', 'Cabin_B4', 'Ticket_F.C.C. 13529', 'Cabin_C2', 'Ticket_2631', 'Ticket_2624', 'Ticket_349213', 'Ticket_111426', 'Ticket_350043', 'Ticket_315096', 'Ticket_231919', 'Cabin_C65', 'Ticket_110564', 'Ticket_315089', 'Ticket_PC 17604', 'Ticket_S.O./P.P. 3', 'Ticket_113806', 'Ticket_229236', 'Cabin_C125']
for i in new_list:
    Xtst_new[i] = 0

In [92]:
final_xtst = Xtst_new[cols_to_keep]

In [93]:
pred = model.predict(final_xtst)

In [98]:
prediction = pd.DataFrame(pred)

In [99]:
prediction = round(prediction,0)

In [100]:
K = B[["PassengerId"]]

In [101]:
K["Survived"] = prediction

In [102]:
K

,PassengerId,Survived
0,892,0.0
1,893,1.0
2,894,0.0
3,895,0.0
4,896,1.0
...,...,...
413,1305,0.0
414,1306,1.0
415,1307,0.0
416,1308,0.0


In [106]:
K.to_csv("C:/Users/kruta/Desktop/submission.csv",index = False)